<a href="https://colab.research.google.com/github/padmasre/sentiment_analysis/blob/main/notebook/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis using Machine Learning

The goal of this project is to train a machine learning model that will predict the sentiment of a movie review as "Positive" or "Negative".

Dataset used to traing the model: [IMDB dataset](https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format?resource=download)

## Setup Kaggle Token

Setup kaggle token to download kaggle dataset needed for this project

In [ ]:
# Creating this folder and file to store kaggle API key to download the dataset
!mkdir -p ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [ ]:
kaggle_username = input("Kaggle Username:")
kaggle_key = input("Kaggle Key:")
api_token = {"username":kaggle_username,"key":kaggle_key}

In [ ]:
import json

with open('/root/.kaggle/kaggle.json', 'w+') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

## Import Required Libraries

In [2]:
import re
import os
import pandas as pd
import pickle
from string import punctuation
from textblob import Word
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Data Loading and Preprocessing

### Data Loading

Downloading dataset from kaggle and loading it into dataframes

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/sentiment_analysis' # Path to store the datasets

In [12]:

train = pd.read_csv(f"{path}/dataset/Train.csv")
test = pd.read_csv(f"{path}/dataset/Test.csv")
validate = pd.read_csv(f"{path}/dataset/Valid.csv")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [ ]:
train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


In [ ]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


### Preprocessing

The `preprocess` function has various text preprocessing steps that include removing HTML tags, tokenization, removing stop words, lemmatization etc.,

The train, test and validation datasets are preprocessed using this function

In [ ]:
def preprocess(df):
    #HTML Tags removal
    df['text'] = df['text'].apply(lambda words: re.sub('<.*?>','',words))
    #Word Tokenization
    df['text'] = df['text'].apply(word_tokenize)
    #Lower case conversion
    df['text'] = df['text'].apply(lambda words: [x.lower() for x in words])
    #Punctuation removal
    df['text'] = df['text'].apply(lambda words: [x for x in words if not x in punctuation])
    #Number removal
    df['text'] = df['text'].apply(lambda words: [x for x in words if not x.isdigit()])
    #Stopword removal
    df['text'] = df['text'].apply(lambda words: [x for x in words if x not in stopwords.words('english')])
    #Frequent word removal
    temp = df['text'].apply(lambda words: " ".join(words))
    freq = pd.Series(temp).value_counts()[:10]
    df['text'] = df['text'].apply(lambda words: [x for x in words if x not in freq.keys()])
    #Lemmatization
    df['text'] = df['text'].apply(lambda words: " ".join([Word(x).lemmatize() for x in words]))
    return df

In [ ]:
train = preprocess(train)
test = preprocess(test)
validate = preprocess(validate)

In [ ]:
train.head()

,text,label
0,grew b watching loving thunderbird mate school...,0
1,put movie dvd player sat coke chip expectation...,0
2,people know particular time past like feel nee...,0
3,even though great interest biblical movie bore...,0
4,im die hard dad army fan nothing ever change g...,1


In [30]:
r = train.iloc[10:11, [0]]
r

,text
10,I can't believe people are looking for a plot ...


In [33]:
r['text'].apply(word_tokenize)

10    [I, ca, n't, believe, people, are, looking, fo...
Name: text, dtype: object

In [ ]:
X_train = train.text
Y_train = train.label
X_validate = validate.text
Y_validate = validate.label
X_test = test.text
Y_test = test.label

## Model Training

Training a logistic regression model using the training dataset.
Leveraging scikit-learn pipeline to build the training pipeline for the model.


Trainine Steps
*   CountVectorizer - Convert a collection of text documents to a matrix of token counts.
*   LogisticRegression - Logistic Regression model



In [ ]:
#Creating a Pipeline
clf = Pipeline(steps =[
('preprocessing', CountVectorizer()),
('classifier', LogisticRegression(max_iter=2000))
])
#Fitting the model
clf.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing', CountVectorizer()),
                ('classifier', LogisticRegression(max_iter=2000))])

In [ ]:
#Calculation model Scores
clf.score(X_valid, Y_valid)

0.8918

In [ ]:
clf.score(X_test,Y_test)

In [ ]:
p = clf.predict(X_test)

In [ ]:
p

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
pickle.dump(clf, open(f'{path}/model/model.pkl', 'wb'))

In [ ]:
print(f'Number of reviews classified as Positive: {list(p).count(1)}')
print(f'Number of reviews classified as Negative: {list(p).count(0)}')

Number of reviews classified as Positive: 2536
Number of reviews classified as Negative: 2464


In [13]:
model = pickle.load(open(f'{path}/model/model.pkl', 'rb'))

In [47]:
predictions = model.predict(['This is a good review', 'This is a good review']).tolist()